# GAN BraTS 2022
* filtre = 24
* n_patch = 1
* lr = dynamic
* alpha = 5

In [ ]:
! pip install elasticdeform
! pip install tensorflow_addons
! pip install gdown

In [ ]:
#IMPORTS
import gdown
import tensorflow as tf
import numpy as np
import tarfile
import nibabel as nib
import glob
import time
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sys import stdout
import matplotlib.pyplot as plt
import matplotlib.image as mpim
from scipy.ndimage.interpolation import affine_transform
from sklearn.model_selection import train_test_split
import json
import numpy as np
from scipy.ndimage.interpolation import affine_transform
import elasticdeform
import multiprocessing as mp
import os
import numpy as np
import tensorflow as tf
import nibabel as nib
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Conv3D, Conv3DTranspose, Dropout, ReLU, LeakyReLU, Concatenate, ZeroPadding3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import tensorflow_addons as tfa
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.layers import BatchNormalization
import numpy as np
from scipy import ndimage
from sklearn.metrics import precision_score
from tqdm import tqdm  
from time import sleep 
import math
from keras import backend as K

In [ ]:
#IMPORT INPUT FOLDER FROM DRIVE
inputs_file = 'https://drive.google.com/file/d/1eSUN5_gtCkpPn277GpLe-Pr_suiYBtWU/view?usp=sharing'
inputs_url = 'https://drive.google.com/uc?id=1eSUN5_gtCkpPn277GpLe-Pr_suiYBtWU'

inputs_output = 'inputs.zip'
gdown.download(inputs_url, inputs_output, quiet=False)

In [ ]:
! unzip ./inputs.zip -d ./INPUTS
! mv -v ./INPUTS/content/drive/MyDrive/GANs/BrainTumorDetection/INPUTS_2021/* ./INPUTS
! rm -rf ./INPUTS/content
! rm ./inputs.zip

In [ ]:
#Eeach execution has 5 epochs 
#kaggle can run 12 hours at a time
EXEC = 59
VERSION = 'brats_22'
INPUT_PATH = './INPUTS'
PATH = './RESULTS' 
DATA_PATH = './data'

In [ ]:
#Import result folder  from previous epoch to continue the training
if EXEC!=1:
    results_file = 'https://drive.google.com/file/d/15mb9qMku7cvycVYrY02Ifvh_LkA_zEqG/view?usp=sharing'
    results_url = 'https://drive.google.com/uc?id=15mb9qMku7cvycVYrY02Ifvh_LkA_zEqG'

    results_output = 'results.zip'

    gdown.download(results_url, results_output, quiet=False)
    
else:
    ! mkdir ./RESULTS

In [ ]:
if EXEC!=1:
    ! unzip ./results.zip -d ./RESULTS
    ! mv -v ./RESULTS/content/drive/MyDrive/GANs/BrainTumorDetection/RESULTS_KAGGLE/VERSION_$VERSION/RESULTS_{EXEC-1}/* ./RESULTS
    ! rm -rf ./RESULTS/content
    ! rm ./results.zip  

In [ ]:
#PATCH EXTRACTION AND AUGMENTATION 

def patch_extraction(Xb, yb, sizePatches=128, Npatches=1, max_tries = 50, background_threshold=0.95, num_classes=4, apply_ratio=False):
    """
    3D patch extraction
    """
    
    batch_size, rows, columns, slices, channels = Xb.shape
    X_patches = np.empty((batch_size*Npatches, sizePatches, sizePatches, sizePatches, channels))
    y_patches = np.empty((batch_size*Npatches, sizePatches, sizePatches, sizePatches))
    i = 0
    list_id = []
    
    if apply_ratio:
      #print(f'Application of ratio(bg_ratio < {background_threshold*100}%): {apply_ratio}')
      for b in range(batch_size):
          for p in range(Npatches):
              
              temp_X = None
              temp_y = None

              tries = 0
              out = False    
              while (tries < max_tries) and (not out):
                #choose a random position from which the patch starts
                x = np.random.randint(rows-sizePatches+1) 
                y = np.random.randint(columns-sizePatches+1)
                z = np.random.randint(slices-sizePatches+1) 

                # extract the brain patch
 
                temp_X = Xb[b, x:x+sizePatches, y:y+sizePatches, z:z+sizePatches, :]

                # extract GT
                temp_y = yb[b, x:x+sizePatches, y:y+sizePatches, z:z+sizePatches]

                # One-hot Encoding
                temp_y_cat = keras.utils.to_categorical(temp_y, num_classes=num_classes)
                
               
                bgrd_ratio = np.sum(temp_y_cat[:, :, :, 0])/(sizePatches * sizePatches * sizePatches)

                tries += 1

                if bgrd_ratio < background_threshold:
                  out = True
                  temp_X = Xb[b, x:x+sizePatches, y:y+sizePatches, z:z+sizePatches, :]

              #print(f'\nBG_ratio : {bgrd_ratio*100} %')
              #print(f"\nTried {tries} times to find a sub-volume. Giving up...")
              X_patches[i] = temp_X
              y_patches[i] = temp_y
              i += 1

    else:
      for b in range(batch_size):
        for p in range(Npatches):
            x = np.random.randint(rows-sizePatches+1) 
            y = np.random.randint(columns-sizePatches+1)
            z = np.random.randint(slices-sizePatches+1) 

            X_patches[i] = Xb[b, x:x+sizePatches, y:y+sizePatches, z:z+sizePatches, :]
            y_patches[i] = yb[b, x:x+sizePatches, y:y+sizePatches, z:z+sizePatches]
            i += 1
                              
    return X_patches, y_patches

def flip3D(X, y):
    """
    Flip the 3D image respect one of the 3 axis chosen randomly
    """
    choice = np.random.randint(3)
    if choice == 0: # flip on x
        X_flip, y_flip = X[::-1, :, :, :], y[::-1, :, :]
    if choice == 1: # flip on y
        X_flip, y_flip = X[:, ::-1, :, :], y[:, ::-1, :]
    if choice == 2: # flip on z
        X_flip, y_flip = X[:, :, ::-1, :], y[:, :, ::-1]
        
    return X_flip, y_flip


def rotation3D(X, y):
    """
    Rotate a 3D image with alfa, beta and gamma degree respect the axis x, y and z respectively.
    The three angles are chosen randomly between 0-30 degrees
    """
    alpha, beta, gamma = np.pi*np.random.random_sample(3,)/6
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(alpha), -np.sin(alpha)],
                   [0, np.sin(alpha), np.cos(alpha)]])
    
    Ry = np.array([[np.cos(beta), 0, np.sin(beta)],
                   [0, 1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    
    Rz = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                   [np.sin(gamma), np.cos(gamma), 0],
                   [0, 0, 1]])
    
    R = np.dot(np.dot(Rx, Ry), Rz)
    
    X_rot = np.empty_like(X)
    for channel in range(X.shape[-1]):
        X_rot[:,:,:,channel] = affine_transform(X[:,:,:,channel], R, offset=0, order=3, mode='constant')
    y_rot = affine_transform(y, R, offset=0, order=0, mode='constant')
    
    del(Rx, Ry, Rz, R)
    return X_rot, y_rot

def brightness(X, y):
    """
    Changing the brighness of a image using power-law gamma transformation.
    Gain and gamma are chosen randomly for each image channel.
    
    Gain chosen between [0.9 - 1.1]
    Gamma chosen between [0.9 - 1.1]
    
    new_im = gain * im^gamma
    """
    
    X_new = np.zeros(X.shape)
    for c in range(X.shape[-1]):
        im = X[:,:,:,c]        
        gain, gamma = (1.2 - 0.8) * np.random.random_sample(2,) + 0.8
        im_new = np.sign(im)*gain*(np.abs(im)**gamma)
        X_new[:,:,:,c] = im_new 
        
    del(im, gain, gamma, im_new)
    return X_new, y

def elastic(X, y):
    """
    Elastic deformation on a image and its target
    """  
    [Xel, yel] = elasticdeform.deform_random_grid([X, y], sigma=2, axis=[(0, 1, 2), (0, 1, 2)], order=[1, 0], mode='constant')
    
    return Xel, yel

def random_decisions(N):
    """
    Generate N random decisions for augmentation
    N should be equal to the batch size
    """
    
    decisions = np.zeros((N, 4)) # 4 is number of aug techniques to combine (patch extraction excluded)
    for n in range(N):
        decisions[n] = np.random.randint(2, size=4)
        
    return decisions

def combine_aug(X, y, do):
    """
    Combine randomly the different augmentation techniques written above
    """
    Xnew, ynew = X, y
    
    # make sure to use at least the 25% of original images
    if np.random.random_sample()>0.75:
        return Xnew, ynew
    
    else:   
        if do[0] == 1:
            Xnew, ynew = flip3D(Xnew, ynew)

        if do[1] == 1:
            Xnew, ynew = brightness(Xnew, ynew)   

        if do[2] == 1:
            Xnew, ynew = rotation3D(Xnew, ynew)

        if do[3] == 1:
            Xnew, ynew = elastic(Xnew, ynew)
        
        return Xnew, ynew

def aug_batch(Xb, Yb):
    """
    Generate a augmented image batch 
    """
    batch_size = len(Xb)
    newXb, newYb = np.empty_like(Xb), np.empty_like(Yb)
    
    decisions = random_decisions(batch_size)            
    inputs = [(X, y, do) for X, y, do in zip(Xb, Yb, decisions)]
    pool = mp.Pool(processes=8)
    multi_result = pool.starmap(combine_aug, inputs)
    pool.close()
    
    for i in range(len(Xb)):
        newXb[i], newYb[i] = multi_result[i][0], multi_result[i][1]
    
    del(decisions, inputs, multi_result, pool)
    return newXb, newYb 

In [ ]:
#load image and data generator 
#generate train set and valid set

def load_img(img_files):
    ''' Load one image and its target form file
    '''
    N = len(img_files)
    # target
    y = nib.load(img_files[N-1]).get_fdata(dtype='float32', caching='unchanged')
    y = y[40:200,34:226,8:136]
    y[y==4]=3
      
    X_norm = np.empty((240, 240, 155, 4))
    for channel in range(N-1):
        X = nib.load(img_files[channel]).get_fdata(dtype='float32', caching='unchanged')
        brain = X[X!=0] 
        brain_norm = np.zeros_like(X) 
        # background at -100
        norm = (brain - np.mean(brain))/np.std(brain)
        brain_norm[X!=0] = norm
        X_norm[:,:,:,channel] = brain_norm        
        
    X_norm = X_norm[40:200,34:226,8:136,:]    
    del(X, brain, brain_norm)
    
    return X_norm, y
    
    
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=4, dim=(160,192,128), n_channels=4, n_classes=4, shuffle=True, augmentation=False, patch_size=128, n_patches=1):
        'Initialization'
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.patch_size = patch_size
        self.n_patches = n_patches
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.list_IDs) // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data     
        X, y = self.__data_generation(list_IDs_temp)
        if self.augmentation == True:
            X, y = self.__data_augmentation(X, y)
        
        if index == self.__len__()-1:
            self.on_epoch_end()
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
  
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim))

        # Generate data
        for i, IDs in enumerate(list_IDs_temp):
            # Store sample
            X[i], y[i] = load_img(IDs)
            
        if self.augmentation == True:
            return X.astype('float32'), y
        else:
            return X.astype('float32'), to_categorical(y, self.n_classes)

    def __data_augmentation(self, X, y):
        'Apply augmentation'
        X_aug, y_aug = patch_extraction(X, y, sizePatches=self.patch_size, Npatches=self.n_patches, apply_ratio=False)
        X_aug, y_aug = aug_batch(X_aug, y_aug)
                
        return X_aug, to_categorical(y_aug, self.n_classes)

In [ ]:
#MODELS Unet3DGan

def double_conv(x, Nf, ks, norm=True):
      for ss in range(2):
          x = Conv3D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
          
          if (norm):
              x = BatchNormalization()(x) 
          x = ReLU()(x) 
      return x   

def Generator():
    '''
    Generator model
    '''
    
    def encoder_step(layer, Nf, ks, norm=True):
        x = double_conv(layer, Nf/2, ks, norm)
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(x)  
        x = Dropout(0.1)(x) 

        return x

    def bottlenek(layer, Nf, ks):
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        
        for i in range(4):
            y = Conv3D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
            x = BatchNormalization()(y)
            x = ReLU()(x)
            x = Concatenate()([x, y])
#             print(x.shape)
        return x

    def decoder_step(layer, layer_to_concatenate, Nf, ks):
        
        x = Dropout(0.1)(layer)
        x = Conv3DTranspose(Nf, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal')(x)
        x = Concatenate()([x, layer_to_concatenate])
        x = double_conv(x, Nf, ks)
        return x

    layers_to_concatenate = []
    inputs = Input((128,128,128,4), name='input_image')
    Nfilter_start = 24
    depth = 5
    ks = 4
    x = inputs

    # encoder
    for d in range(depth):
        if d==0: 
          x = Conv3D(Nfilter_start, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
          x = Conv3D(Nfilter_start, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
        else:
          x = encoder_step(x, Nfilter_start*np.power(2,d), ks)
        
        layers_to_concatenate.append(x)

    # bottlenek
    x = bottlenek(x, Nfilter_start*np.power(2,depth-1), ks)
    
    # decoder
    for d in range(depth-2, -1, -1): 
        x = decoder_step(x, layers_to_concatenate.pop(), Nfilter_start*np.power(2,d), ks)
    
    # classifier
    last = Conv3DTranspose(4, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal', activation='softmax', name='output_generator')(x)
    
    del(x,layers_to_concatenate)
    return Model(inputs=inputs, outputs=last, name='Generator')

def Discriminator():
    '''
    Discriminator model
    '''

    inputs = Input((128,128,128,4), name='input_image')
    targets = Input((128,128,128,4), name='target_image')
    Nfilter_start = 24
    depth = 3
    ks = 4

    def encoder_step(layer, Nf, norm=True):
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        if norm:
            x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.1)(x)
        
        return x

    x = Concatenate()([inputs, targets])

    for d in range(depth+1):
        if d==0:
            x = encoder_step(x, Nfilter_start*np.power(2,d), False)
        else:
            x = encoder_step(x, Nfilter_start*np.power(2,d))
            

    last = Conv3D(1, ks, strides=1, padding='valid', kernel_initializer='he_normal', name='output_discriminator')(x) 
    
    del(x)
    return Model(inputs=[targets, inputs], outputs=last, name='Discriminator')

def ensembler():

    start = Input((128,128,128,40))
    fin = Conv3D(4, kernel_size=3, kernel_initializer='he_normal', padding='same', activation='softmax')(start)

    return Model(inputs=start, outputs=fin, name='Ensembler')
    


In [ ]:
#calcul losses


def diceLoss(y_true, y_pred, class_weights):
    y_true = tf.convert_to_tensor(y_true, 'float32')
    y_pred = tf.convert_to_tensor(y_pred, y_true.dtype)

    num = tf.math.reduce_sum(tf.math.multiply(class_weights, tf.math.reduce_sum(tf.math.multiply(y_true, y_pred), axis=[0,1,2,3])))
    den = tf.math.reduce_sum(tf.math.multiply(class_weights, tf.math.reduce_sum(tf.math.add(y_true, y_pred), axis=[0,1,2,3])))+1e-5
    return 1-2*num/den

def discriminator_loss(disc_real_output, disc_fake_output):
    real_loss = tf.math.reduce_mean(tf.math.pow(tf.ones_like(disc_real_output) - disc_real_output, 2))
    fake_loss = tf.math.reduce_mean(tf.math.pow(tf.zeros_like(disc_fake_output) - disc_fake_output, 2))

    disc_loss = 0.5*(real_loss + fake_loss)

    return disc_loss


def generator_loss(target, gen_output, disc_fake_output, class_weights, alpha):
    
    # generalized dice loss
    dice_loss = diceLoss(target, gen_output, class_weights)
    
    # disc loss
    disc_loss = tf.math.reduce_mean(tf.math.pow(tf.ones_like(disc_fake_output) - disc_fake_output, 2))
   
    # total loss
    gen_loss = alpha*dice_loss + disc_loss
   

    return gen_loss, dice_loss, disc_loss



# *  **WT = ET ∪ ED ∪ NCR/NET**
# *  **TC = ET ∪ NCR/NET**
# *  **ET = ET**



1.   ED (edema -> classe 2)
2.   ET (enhancing_tumor -> classe 3)
3.   NCR/NET (non_enhancing_tumor -> classe 1)
4.   BG (background -> classe 0)







In [ ]:
#METRICS EVALUATING



def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    smooth = 1e-5

    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def binary_dice3d(s,g):
    #dice score of two 3D volumes
    smooth = 1e-5
    num=np.sum(np.multiply(s, g))
    denom=s.sum() + g.sum() + smooth
    
    return  2.0*num/denom

def sensitivity (seg,ground): 
    #computs false negative rate
    smooth = 1e-5
    num=np.sum(np.multiply(ground, seg))
    denom=np.sum(ground)+smooth
    
    return  num/denom

def specificity (seg,ground): 
    #computes false positive rate
    smooth = 1e-5
    num=np.sum(np.multiply(ground==0, seg ==0))
    denom=np.sum(ground==0)+smooth
    
    return  num/denom

def border_map(binary_img,neigh):
    """
    Creates the border for a 3D image
    """
    binary_map = np.asarray(binary_img, dtype=np.uint8)
    neigh = neigh
    west = ndimage.shift(binary_map, [-1, 0,0], order=0)
    east = ndimage.shift(binary_map, [1, 0,0], order=0)
    north = ndimage.shift(binary_map, [0, 1,0], order=0)
    south = ndimage.shift(binary_map, [0, -1,0], order=0)
    top = ndimage.shift(binary_map, [0, 0, 1], order=0)
    bottom = ndimage.shift(binary_map, [0, 0, -1], order=0)
    cumulative = west + east + north + south + top + bottom
    border = ((cumulative < 6) * binary_map) == 1
    return border

def border_distance(ref,seg):
    """
    This functions determines the map of distance from the borders of the
    segmentation and the reference and the border maps themselves
    """
    neigh=8
    border_ref = border_map(ref,neigh)
    border_seg = border_map(seg,neigh)
    oppose_ref = 1 - ref
    oppose_seg = 1 - seg
    # euclidean distance transform
    distance_ref = ndimage.distance_transform_edt(oppose_ref)
    distance_seg = ndimage.distance_transform_edt(oppose_seg)
    distance_border_seg = border_ref * distance_seg
    distance_border_ref = border_seg * distance_ref
    return distance_border_ref, distance_border_seg#, border_ref, border_seg

def Hausdorff_distance(ref,seg):
    """
    This functions calculates the average symmetric distance and the
    hausdorff distance between a segmentation and a reference image
    :return: hausdorff distance and average symmetric distance
    """
    ref_border_dist, seg_border_dist = border_distance(ref,seg)
    hausdorff_distance = np.max([np.max(ref_border_dist), np.max(seg_border_dist)])
    return hausdorff_distance

def DSC_whole(pred, orig_label):
    #computes dice for the whole tumor
    return dice_coef(pred>0,orig_label>0)

def DSC_en(pred, orig_label):
    #computes dice for enhancing region
    return dice_coef(pred==3,orig_label==3)

def DSC_core(pred, orig_label):
    #computes dice for core region
    seg_=np.copy(pred)
    ground_=np.copy(orig_label)
    seg_[seg_==2]=0
    ground_[ground_==2]=0
    return dice_coef(seg_>0,ground_>0)

def sensitivity_whole (seg,ground):
    return sensitivity(seg>0,ground>0)

def sensitivity_en (seg,ground):
    return sensitivity(seg==3,ground==3)

def sensitivity_core (seg,ground):
    seg_=np.copy(seg)
    ground_=np.copy(ground)
    seg_[seg_==2]=0
    ground_[ground_==2]=0
    return sensitivity(seg_>0,ground_>0)

def specificity_whole (seg,ground):
    return specificity(seg>0,ground>0)

def specificity_en (seg,ground):
    return specificity(seg==3,ground==3)

def specificity_core (seg,ground):
    seg_=np.copy(seg)
    ground_=np.copy(ground)
    seg_[seg_==2]=0
    ground_[ground_==2]=0
    return specificity(seg_>0,ground_>0)
    
def hausdorff_whole (seg,ground):
    return Hausdorff_distance(seg==0,ground==0)

def hausdorff_en (seg,ground):
    return Hausdorff_distance(seg!=3,ground!=3)

def hausdorff_core (seg,ground):
    seg_=np.copy(seg)
    ground_=np.copy(ground)
    seg_[seg_==2]=0
    ground_[ground_==2]=0
    return Hausdorff_distance(seg_==0,ground_==0)

In [ ]:

class_weights = np.load(INPUT_PATH + '/class_weights.npy')

# Models
G = Generator()
D = Discriminator()

if os.path.exists(PATH + f'/{VERSION}_Generator.h5')==True and os.path.exists(PATH + f'/{VERSION}_Discriminator.h5')==True: 
    G.load_weights(PATH + f'/{VERSION}_Generator.h5')
    D.load_weights(PATH + f'/{VERSION}_Discriminator.h5')

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.5)



@tf.function
def train_step(image, target, alpha):
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

      gen_output = G(image, training=True)
      disc_real_output = D([image, target], training=True)
      disc_fake_output = D([image, gen_output], training=True)
      disc_loss = discriminator_loss(disc_real_output, disc_fake_output)
      
      gen_loss, dice_loss, disc_loss_gen = generator_loss(target, gen_output, disc_fake_output, class_weights, alpha)
      del gen_output, disc_fake_output,image
  generator_gradients = gen_tape.gradient(gen_loss, G.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss, D.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, G.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, D.trainable_variables))
      
  return gen_loss, dice_loss, disc_loss_gen
        


def fit(train_gen, valid_gen, alpha, epochs, jump):
    
    if os.path.exists(PATH)==False:
        os.mkdir(PATH)
        
    Nt = len(train_gen)
    history = {'train': [], 'valid': []}
    
    stats={'dice':[],'spec':[],'sen':[],'hau95':[]}
    prev_loss = np.inf
    
    epoch_ugan_loss = tf.keras.metrics.Mean()
    epoch_dice_loss = tf.keras.metrics.Mean()
    epoch_disc_loss = tf.keras.metrics.Mean()
    
    
    ugan_min = 2
    
    for e in range(epochs):
        print('Epoch {}/{}'.format(e+1,epochs))
        b = 0
        
        x=(EXEC -1)*EPOCHS + e
        lrg=(math.exp(-x))*0.001 + 0.00001
        lrd=(math.exp(-x))*0.001 + 0.00002
        K.set_value(generator_optimizer.learning_rate, lrg)
        K.set_value(discriminator_optimizer.learning_rate, lrd)
        
        for Xb, yb in tqdm(train_gen):

          b += 1 
          losses = train_step(Xb, yb, alpha)
          epoch_ugan_loss.update_state(losses[0])
          epoch_dice_loss.update_state(losses[1])
          epoch_disc_loss.update_state(losses[2])
          
          print('\n Batch: {}/{} - loss: {:.4f} - dice_loss: {:.4f} - disc_loss: {:.4f}'
                        .format(b, Nt, epoch_ugan_loss.result(), epoch_dice_loss.result(), epoch_disc_loss.result()))
          
#           stdout.flush()
        del Xb,yb    
        history['train'].append([epoch_ugan_loss.result(), epoch_dice_loss.result(), epoch_disc_loss.result()])
        
       
       
        try :
           
                G.save_weights(PATH + f'/{VERSION}_Generator.h5') 
                D.save_weights(PATH + f'/{VERSION}_Discriminator.h5')
                
        except:
            print('Weights not saved')

        
        epoch_ugan_loss.reset_states()
        epoch_dice_loss.reset_states()
        epoch_disc_loss.reset_states()
        
        
       
        
    return history

In [ ]:
!mkdir data

In [ ]:
zip_file = tarfile.open("../input/brats-2021-task1/BraTS2021_Training_Data.tar")
zip_file.extractall(DATA_PATH)
zip_file.close()

In [ ]:
t1_list = sorted(glob.glob(DATA_PATH+'/*/*_t1.nii.gz'))
t2_list = sorted(glob.glob(DATA_PATH+'/*/*_t2.nii.gz'))
t1ce_list = sorted(glob.glob(DATA_PATH+'/*/*_t1ce.nii.gz'))
flair_list = sorted(glob.glob(DATA_PATH+'/*/*_flair.nii.gz'))
seg_list = sorted(glob.glob(DATA_PATH+'/*/*_seg.nii.gz'))

In [ ]:
# print(len(t1_list),len(t2_list),len(t1ce_list),len(flair_list),len(seg_list))

In [ ]:
#*******************MAIN**********************************************************
#*********************************************************************************
#EXTRACTION OF DATA , T1 T2 T1C TFLAIR GROUNDTRUE
BATCH_SIZE = 4
ALPHA = 5
EPOCHS = 5
NB_CLASSES = 4
N_PATCHES = 1

JUMP = False
pred_img = (PATH + f'/{VERSION}'+'_pred@epoch_{:03d}.png').format((EXEC-1)*EPOCHS) #0,8,16
if os.path.exists(pred_img)==True:
    JUMP = True

# create the training and validation sets or load it from inputs
#Added here
Nim = len(t1_list)

idx = np.arange(Nim)
sets = {'train': [], 'valid': []}
#Train in 100% of data to participate to BraTS witch provide validation data without GT
for i in range(len(t1_list)):
    sets['train'].append([t1_list[i], t2_list[i], t1ce_list[i], flair_list[i], seg_list[i]])


del(t1_list,t2_list,t1ce_list,flair_list,seg_list,idx,Nim)

train_gen = DataGenerator(sets['train'], batch_size=BATCH_SIZE, n_classes=NB_CLASSES, n_patches=N_PATCHES, augmentation=True)
# valid_gen = DataGenerator(sets['valid'], batch_size=BATCH_SIZE, n_classes=NB_CLASSES, n_patches=N_PATCHES, augmentation=True)



In [ ]:
print(len(sets['train']))

# Train the uGAN model

In [ ]:
%%capture cap --no-stderr
h = fit(train_gen, 0, ALPHA, EPOCHS, JUMP) 

In [ ]:
with open(PATH + f'/{VERSION}_fit_output_{EXEC}.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
%%capture cap_h --no-stderr
print(h)

In [ ]:
with open(PATH + f'/{VERSION}_h_{EXEC}.txt', 'w') as f:
    f.write(cap_h.stdout)

In [ ]:
! zip -r './results.zip' {PATH+'/*'}

In [ ]:
from IPython.display import FileLink
FileLink(r'./results.zip')
# then in colab : !wget "https://....kaggle.net/...../results.zip"